In [1]:
import warnings
warnings.filterwarnings("ignore")
import os
import numpy as np
import rasterio
import tqdm
import pandas as pd
import rasterio
import random
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_fscore_support

In [10]:
path_data = "/home/gt/DATA/geolifeclef-2025"
train_path_sentinel = os.path.join(path_data, "SatelitePatches/PA-train")
train_path_landsat = os.path.join(path_data, "SateliteTimeSeries-Landsat/cubes/PA-train")
train_path_bioclim = os.path.join(path_data, "BioclimTimeSeries/cubes/PA-train")
train_metadata = pd.read_csv(os.path.join(path_data, "GLC25_PA_metadata_train.csv"))
train_metadata = train_metadata.dropna(subset="speciesId").reset_index(drop=True)
train_metadata['speciesId'] = train_metadata['speciesId'].astype(int)
train_label_dict = train_metadata.groupby('surveyId')['speciesId'].apply(list).to_dict()
train_metadata = train_metadata.drop_duplicates(subset="surveyId").reset_index(drop=True).drop('speciesId', axis=1)
train_metadata['areaInM2'].fillna((train_metadata['areaInM2'].mean()), inplace=True)
train_elevation = pd.read_csv(os.path.join(path_data, "EnvironmentalValues", "Elevation", "GLC25-PA-train-elevation.csv"))
train_elevation['Elevation'].fillna((train_elevation['Elevation'].mean()), inplace=True)
train_soil = pd.read_csv(os.path.join(path_data, "EnvironmentalValues", "SoilGrids", "GLC25-PA-train-soilgrids.csv"))
meta_columns = ["areaInM2", "Elevation"]

In [14]:
train_elevation = pd.read_csv(os.path.join(path_data, "EnvironmentalValues", "Elevation", "GLC25-PA-train-elevation.csv"))
train_elevation['Elevation'].fillna((train_elevation['Elevation'].mean()), inplace=True)
print(train_elevation.shape, "\n", train_elevation.isna().sum(0))

(88987, 2) 
 surveyId     0
Elevation    0
dtype: int64


In [15]:
train_soil = pd.read_csv(os.path.join(path_data, "EnvironmentalValues", "SoilGrids", "GLC25-PA-train-soilgrids.csv"))
for column in train_soil.columns: train_soil[column].fillna((train_soil[column].mean()), inplace=True)
print(train_soil.shape, "\n", train_soil.isna().sum(0))

(88987, 10) 
 surveyId             0
Soilgrid-bdod        0
Soilgrid-cec         0
Soilgrid-cfvo        0
Soilgrid-clay        0
Soilgrid-nitrogen    0
Soilgrid-phh2o       0
Soilgrid-sand        0
Soilgrid-silt        0
Soilgrid-soc         0
dtype: int64


In [19]:
combined = pd.concat([train_metadata, train_elevation.Elevation, train_soil.drop("surveyId", axis=1)], axis=1)
print(combined.isna().sum(0))
combined

lon                    0
lat                    0
year                   0
geoUncertaintyInM    521
areaInM2               0
region                 0
country                0
surveyId               0
Elevation              0
Soilgrid-bdod          0
Soilgrid-cec           0
Soilgrid-cfvo          0
Soilgrid-clay          0
Soilgrid-nitrogen      0
Soilgrid-phh2o         0
Soilgrid-sand          0
Soilgrid-silt          0
Soilgrid-soc           0
dtype: int64


,lon,lat,year,geoUncertaintyInM,areaInM2,region,country,surveyId,Elevation,Soilgrid-bdod,Soilgrid-cec,Soilgrid-cfvo,Soilgrid-clay,Soilgrid-nitrogen,Soilgrid-phh2o,Soilgrid-sand,Soilgrid-silt,Soilgrid-soc
0,3.099038,43.134956,2021,5.0,100.0,MEDITERRANEAN,France,212,47.0,140.0,214.0,151.0,292.0,159.0,73.0,284.0,422.0,176.0
1,9.884560,56.912140,2017,10.0,79.0,CONTINENTAL,Denmark,222,6.0,120.0,225.0,101.0,94.0,379.0,58.0,650.0,255.0,609.0
2,8.256020,55.637050,2019,10.0,79.0,ATLANTIC,Denmark,243,10.0,115.0,261.0,86.0,131.0,373.0,54.0,750.0,117.0,672.0
3,-0.402590,43.505630,2018,1.0,-inf,ATLANTIC,France,324,116.0,118.0,179.0,117.0,225.0,230.0,57.0,238.0,535.0,302.0
4,-0.517360,45.806430,2017,1.0,-inf,ATLANTIC,France,333,84.0,138.0,233.0,176.0,328.0,250.0,71.0,231.0,439.0,257.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88982,10.327990,57.305850,2018,10.0,79.0,CONTINENTAL,Denmark,3919553,23.0,116.0,199.0,65.0,116.0,311.0,59.0,622.0,260.0,485.0
88983,15.003900,55.090170,2017,10.0,707.0,CONTINENTAL,Denmark,3919592,138.0,118.0,245.0,113.0,156.0,405.0,54.0,555.0,288.0,868.0
88984,8.935060,55.461500,2018,10.0,79.0,ATLANTIC,Denmark,3919620,37.0,113.0,233.0,70.0,71.0,253.0,56.0,798.0,129.0,414.0
88985,17.252948,53.901434,2021,5.0,25.0,CONTINENTAL,Poland,3919640,131.0,130.0,162.0,99.0,55.0,234.0,50.0,803.0,141.0,502.0


In [22]:
meta_columns = ["areaInM2", "Elevation"] + list(train_soil.columns[1:])
meta_columns

['areaInM2',
 'Elevation',
 'Soilgrid-bdod',
 'Soilgrid-cec',
 'Soilgrid-cfvo',
 'Soilgrid-clay',
 'Soilgrid-nitrogen',
 'Soilgrid-phh2o',
 'Soilgrid-sand',
 'Soilgrid-silt',
 'Soilgrid-soc']

In [13]:
train_metadata.value_counts(["areaInM2", "country"])

areaInM2  country    
 79.0     Denmark        42647
-inf      France         10258
 707.0    Denmark         6697
 4.0      Netherlands     4422
 9.0      Netherlands     2195
                         ...  
 90.0     Spain              1
 10.0     Norway             1
 90.0     Italy              1
          Germany            1
 8000.0   Netherlands        1
Name: count, Length: 866, dtype: int64

In [25]:
train_metadata = pd.read_csv(os.path.join(path_data, "GLC25_PA_metadata_train.csv"))
train_metadata.groupby('surveyId')['speciesId'].apply(list)[212]

[6874.0,
 476.0,
 11157.0,
 8784.0,
 4530.0,
 10520.0,
 9458.0,
 982.0,
 51.0,
 3935.0,
 11193.0,
 2421.0,
 2159.0,
 9030.0,
 2932.0,
 1304.0]

In [ ]:
print(train_dataset[0][1].shape)
#val = np.zeros([len(train_dataset)])
#for i, d in enumerate(tqdm.tqdm(train_dataset)):
#    val[i] = torch.sum(torch.sum(torch.reshape(torch.permute(d[1], [0,2,1]), [6, -1]), -2) == 0).numpy()

In [ ]:
train_metadata.speciesId.drop_duplicates().sort_values()
prev_min = 10
train_metadata.value_counts("speciesId").sort_index().pipe(lambda x: x[x>=prev_min])

In [ ]:
plt.plot(np.log10(train_metadata.value_counts("speciesId").values))

In [ ]:
np.sort(train_metadata.value_counts("speciesId").index[:1000].values.astype(int))

In [ ]:
(train_metadata.country.value_counts().cumsum() / test_metadata.shape[0]).index[:10].values

In [ ]:
test_metadata.country.isin(most_train_countries).sum()